# Segmenting and Clustering Neighborhoods in Toronto
## Problem 2

Import necessary libraries

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Grabbing the webcontent

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

Convert the postal code as list and transform the list into a dataframe:

In [14]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head()


,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


As we see there are "NONE" and "Not assigned" values in rows, we need to drop rows:

In [15]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


1. Group by 'PostalCode' and 'Borough'
2. Convert the grouped neighborhoods as string with commas as separators
3. Back to a new dataframe

In [16]:
def nb_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
dfgroup = grp.apply(nb_list).reset_index(name='Neighborhood')
dfgroup.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Reading geospatial data from: http://cocl.us/Geospatial_data

In [22]:
geo = pd.read_csv("http://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now between dfgroup and geo, we have PostalCode in common:
We join the two dataframes on PostalCode

In [ ]:
result = pd.concat([newDF, geoDF], axis=1, )